# UI

> A chat interface UI demo

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.all import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from fastcore.utils import *
import nbformat
import itertools
from functools import wraps

### Initial setup

In [ ]:
custom_highlighting = " ".join(name for name in globals() if name[0].isupper())
ftrs = CodeHighlightThemeScript(custom_highlighting)

In [ ]:
app = FastHTML(ftrs=ftrs, exts=("ws", "preload"), session_cookie="mysession")
rt = app.route

In [ ]:
@app.middleware("http")
async def add_cache_headers(request, call_next):
    response = await call_next(request)
    if "/documentation-rag/" not in str(request.url):
        response.headers["Cache-Control"] = "max-age=300"  # 5 minutes
    return response

In [ ]:
srv = JupyUvi(app)

In [ ]:
slug_map = {}

In [ ]:
def make_key():
    current = [None]
    def key_fn(cell):
        if cell.cell_type == "markdown" and cell.source.startswith("###"):
            current[0] = cell.source.strip("###").strip()
        return current[0]
    return key_fn

In [ ]:
def mk_nbdict(filename:str):
    nb = nbformat.read(filename, as_version=4)
    cells = itertools.islice(itertools.dropwhile(lambda c: c.source != "# Components", nb.cells), 1, None)
    nbdict = {}
    for header, group in itertools.groupby(cells, key=make_key()):
        header_slug = slugify(header)
        slug_map[header_slug] = header
        nbdict[header_slug] = list(group)
    return nbdict 


In [ ]:
mk_nbdict("03_chat.ipynb").keys()

dict_keys(['chat-input', 'chat-message', 'chat-code', 'chat-pairs'])

In [ ]:
matched = re.search(r"\d+_(\w+)\.ipynb", "01_common.ipynb").group(1)
matched

'common'

In [ ]:
def format_nm(nm:str):
    return re.search(r"\d+_(\w+)\.ipynb", nm).group(1).title()

In [ ]:
documentation = {format_nm(nm): mk_nbdict(nm) for nm in ["01_common.ipynb", "02_interactive.ipynb", "03_chat.ipynb"]}

In [ ]:
documentation.keys()

dict_keys(['Common', 'Interactive', 'Chat'])

### Search bar

In [ ]:
def SearchBarButton(id):
    return DialogOpenButton(
        Div(
            Div(
                Icon("search", cls="size-4"),
                Span("Search Documentation", cls="text-sm"),
                cls="flex gap-2 items-center",
            ),
            Kbd("⌘ K", cls="text-2xs"),
            cls="flex items-center px-3 py-2 justify-between",
        ),
        cls="m-1 hover:cursor-pointer bg-background rounded-lg w-full",
        did=id,
    )

In [ ]:
def mk_command_items(category:str, nms:list[str]):
    return [
        A(
            CommandItem(slug_map[nm]), 
            href=f"/{category.lower()}/{nm}",
            cls="block w-full",
            onclick="this.closest('dialog').close(); document.activeElement.blur()"
        )
        for nm in nms
    ]

In [ ]:
def SearchBarCommand(id):
    groups = [
        CommandGroup(
            ItemHeader(category), 
            *mk_command_items(category, nbdict.keys())
        )
        for category, nbdict in documentation.items()
    ]
    
    return CommandDialog(
        Command(
            CommandSearch(),
            CommandScrollable(*groups, style="height: 400px;"),
            cls="w-4xl",
            style="top: 29%; max-width: 650px; max-height: 500px;",

        ),
        id=id,


    )

In [ ]:
def SearchBar(id="search-bar"):
    return Div(
        SearchBarButton(id=id),
        SearchBarCommand(id=id),
        Script(f"""
        document.addEventListener('keydown', (e) => {{
            if ((e.ctrlKey || e.metaKey) && e.key === 'k') {{
                e.preventDefault();
                const dialog = document.getElementById("{id}");
                if (dialog.open) {{
                    dialog.close();
                }} else {{
                    dialog.showModal();
                    dialog.querySelector("header input")?.focus();
                }}
            }}
        }});
        """),
        cls="flex w-full",
    )

### Sidebar

In [ ]:
def DropdownFooter():
    return Div(
            Dropdown(
                DropdownItem("Upgrade to Pro"),
                Separator(),
                DropdownItem("Account"),
                trigger_btn=DropdownTriggerButton(
                    Div(
                        Avatar(src="https://ui.shadcn.com/avatars/shadcn.jpg"), Span("Taya", cls="pl-2"),
                        cls="flex items-center w-full",
                    ),
                    Icon("chevrons-up-down"), did="sidebar-dropdown", 
                    cls=f"{ButtonT.ghost} justify-between py-6 px-3 m-1",
                    style="width: calc(100% - 0.5rem)"
                ),
                id="sidebar-dropdown",
                side="right", align="end",
        ),
        cls="flex flex-col items-left w-full",
    )

In [ ]:
category_icons = {"Common": "blocks", "Interactive": "mouse-pointer-click", "Chat": "message-square"}

In [ ]:
def SiteSidebar():
    contents = [
        SidebarCollapsable(
            IconTitle(category, icon=category_icons.get(category)), 
            [slug_map[k] for k in nbdict.keys()],
            href_list=nbdict.keys(),
            href_parent=category,
        ) 
        for category, nbdict in documentation.items()
    ]
    return Sidebar(
        SearchBar(),
        Group(
            "Components", 
            *contents
        ),
        SidebarGroup("Demos", ["Documentation RAG"], icon_list=["bot"], href_parent="demos"),
        footer=DropdownFooter(),
        hx_boost="true", hx_target="#main-content", hx_select="#main-content", hx_ext="preload", preload="mouseover",
    )

In [ ]:
def TopBar(category="", component=""):
    return Div(
        Div(
            ToggleButton(),
            SeparatorVertical(),
            Breadcrumb([category.title(), slug_map.get(component, component.title())]),
            cls="flex items-center gap-2"
        ),
        ThemeSwitcher(),
        cls="flex justify-between items-center w-full px-3 overflow-hidden py-1 border-b"
    )

In [ ]:
def sidebar_layout(fn):
    @wraps(fn)
    def inner_fn(*args, **kwargs):
        return Div(
            SiteSidebar(),
            Main(
                TopBar(**kwargs),
                Div(
                    fn(*args, **kwargs),
                    cls="flex flex-col flex-1 overflow-y-auto min-h-0",
                ),
                cls="h-full flex flex-col",
                id="main-content"
            ),
            cls="h-screen w-full",
        )
    return inner_fn

### Chat interface

In [ ]:
from lisette import *
chat = Chat(model="anthropic/claude-3-haiku-20240307")
chat

<lisette.core.Chat>

In [ ]:
# for r in chat("Testing2", stream=True):
#     delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
#     if delta and delta.content:
#         print(delta.content, end='', flush=True)
# final_response = r.choices[0].message.content

In [ ]:
@app.ws("/send_msg_ws")
async def send_msg_ws(text:str, send):
    await send(Div(ChatPrompt(text), id="chat-interface", hx_swap_oob="beforeend"))
    r = chat(text, stream=False)
    output = r.choices[0].message.content
    await send(Div(ChatAssistant(output), id="chat-interface", hx_swap_oob="beforeend"))

In [ ]:
def RAGChat():
    return ChatInterface(
            ChatPrompt("A test message sent by a user\n\nAnd new lines\nAre here."),
            ChatAssistant("A smaller response returned by a LLM. Great question. You are absolutely right!"),
            chat_input=ChatInput(
                cls="mb-1 mt-0", ta_cls="rounded-t-none", width="w-full",
                hx_ext="ws", ws_connect="/send_msg_ws", ws_send=True,
            ),
            cls="flex-1",
            id="chat-interface",
        )

In [ ]:
@rt("/demos/documentation-rag")
@sidebar_layout
def documentation_rag():
    return Div(
        RAGChat(),
        cls="flex flex-1",
    )

### Dyanmic routes

#### Display demo card

In [ ]:
def find_demos(nb_cell_list:list):
    return L(nb_cell_list).filter(Self.source.startswith("#| demo"))

In [ ]:
def exec_demo(cell:dict):
    ns = {}
    exec(cell.source, globals(), ns)
    name, fn = next(iter(ns.items())) 
    return (name, fn, cell.source)

In [ ]:
def render_demo_cards(nb_cell_list):
    return [DisplayDemoCard(*exec_demo(d)) for d in find_demos(nb_cell_list)]

In [ ]:
def DisplayDemoCard(nm, fn, src):
    return Div(
        Tabs(
            contents=[
                CardOutline(fn(), cls="rounded-lg min-h-96 flex justify-center items-center py-5"),
                Card(
                    Pre(
                        Code(src, cls="language-python-custom", style="background: transparent; overflow: visible;"),
                    ), 
                cls="rounded-lg min-h-96 flex text-sm overflow-auto overflow-y-auto",
                ),
            ],
            tablist=["preview", "code"],
            id=f"{nm}-tab",
            cls="w-full",
            nav_cls="w-fit ml-auto",
        ),
        cls="w-full",
    )

#### Source code accordion

In [ ]:
def strip_meta(text, prefixes=["#| exports", "#| export", "\n"]):
    for prefix in prefixes:
        text = text.removeprefix(prefix)
    return text

In [ ]:
def find_sourcecode(nb_cell_list):
    return L(nb_cell_list).filter(Self.source.startswith(("#| demo", "#| preview")), negate=True)[1:]

In [ ]:
def get_code_and_output(cell):
    code = strip_meta(cell["source"])
    output = [strip_meta(get_output_text(o)) for o in cell["outputs"]]
    return code, output

In [ ]:
def get_clean_output(output):
    if not isinstance(output, dict): return str(output)
    data = output.get("data", output)
    text = data.get("text/plain") or data.get("text/html") or data.get("text", "")
    return strip_meta(text)

In [ ]:
def render_cell(c):
    if c["cell_type"] == "markdown": return ChatNote(c["source"])
    code = strip_meta(c["source"])
    out = get_clean_output(c["outputs"][0]) if c["outputs"] else None
    return ChatCodeAndOutput(code, out)

In [ ]:
def AccordionSourceCode(nb_cell_list):
    contents = find_sourcecode(nb_cell_list).map(render_cell)
    return Accordion(
        AccordionItem(
            "Source code",
            ChatInterface(
                *contents,
                inner_cls="border rounded-lg",
            )
        ),
        cls="mt-4"
    )

#### Default component route

In [ ]:
@rt("/{category}/{component}")
@sidebar_layout
def get(category: str, component: str):
    nb_cell_list = documentation.get(category.title(), {}).get(component, {})
    if not nb_cell_list:
        return Div(H3("This page doesn't exist yet...", cls="text-xl mt-10"), cls="flex justify-center")
    return Div(
        H3(slug_map[component], cls="text-xl font-semibold tracking-tight mt-3"),
        Div(*render_demo_cards(nb_cell_list), cls="flex flex-col gap-3"),
        AccordionSourceCode(nb_cell_list),
        cls="flex flex-col w-full lg:max-w-4xl mx-auto",
    )

### Index

In [ ]:
def CopyableCode(code, lang="bash"):
    return Div(
        Div(Span(lang, cls="text-xs text-muted-foreground"), cls="px-4 py-2 border-b"),
        Div(
            # Pre(Code(code, cls=f"language-{lang}"), cls="p-4 overflow-x-auto"),
            CodeHighlight(code),
            # Button(Icon("copy", cls="size-4"), cls=f"{ButtonT.ghost} absolute top-2 right-2 size-8 p-0"),
            Button(
                Icon("copy", cls="size-4 copy-icon"),
                Icon("check", cls="size-4 check-icon hidden text-green-500"),
                cls=f"{ButtonT.ghost} absolute top-2 right-2 size-8 p-0",
                onclick="""
                    navigator.clipboard.writeText(this.closest('.relative').querySelector('code').textContent);
                    this.querySelector('.copy-icon').classList.add('hidden');
                    this.querySelector('.check-icon').classList.remove('hidden');
                    setTimeout(() => {
                        this.querySelector('.copy-icon').classList.remove('hidden');
                        this.querySelector('.check-icon').classList.add('hidden');
                    }, 1500);
                """
            ),
            cls="relative"
        ),
        cls="rounded-lg border bg-muted/50"
    )

For the lang part, it makes a bit of a top border along the full top. Is there some way for it to be just around the bash words, then it sort of curves into the border underneat it. So it's like there is a staight table, and then a sort of circular bump along one end?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! You can create that "tab" effect where the language label looks like it's popping out of the top border. Here's the approach:

```python
def CopyableCode(code, lang="bash"):
    return Div(
        Div(
            Span(lang, cls="text-xs text-muted-foreground px-3 py-1 border border-b-0 rounded-t-lg bg-background"),
            cls="px-4 pt-2"
        ),
        Div(
            CodeHighlight(code),
            Button(
                Icon("copy", cls="size-4 copy-icon"),
                Icon("check", cls="size-4 check-icon hidden text-green-500"),
                cls=f"{ButtonT.ghost} absolute top-2 right-2 size-8 p-0",
                onclick="""
                    navigator.clipboard.writeText(this.closest('.relative').querySelector('code').textContent);
                    this.querySelector('.copy-icon').classList.add('hidden');
                    this.querySelector('.check-icon').classList.remove('hidden');
                    setTimeout(() => {
                        this.querySelector('.copy-icon').classList.remove('hidden');
                        this.querySelector('.check-icon').classList.add('hidden');
                    }, 1500);
                """
            ),
            cls="relative border rounded-lg"
        ),
        cls="rounded-lg bg-muted/50"
    )
```

The key changes are putting a border and rounded top corners on just the `Span` containing the language name, and removing the bottom border (`border-b-0`) so it connects smoothly to the code area below.

Does this create the tab effect you're looking for?

In [ ]:
def PopoverFastHTML():
    return Div(
        # H3("FastHTML", cls="text-lg font-semibold"),
        Img(src="https://www.fastht.ml/docs/logo.svg"),
        P("A Python framework for building modern web applications using pure Python — no JavaScript required.", cls="text-muted-foreground text-sm mt-2"),
        Div(
            A("Docs", href="https://docs.fastht.ml", cls="text-primary underline text-sm"),
            A("GitHub", href="https://github.com/AnswerDotAI/fasthtml", cls="text-primary underline text-sm"),
            cls="flex gap-4 mt-3"
        ),
    )

In [ ]:
def UnderlineText():
    return P(
        Span("FastHTML", cls="relative after:content-[''] after:absolute after:bottom-0 after:left-1/2 after:-translate-x-1/2 after:h-0.5 after:w-4 after:bg-blue-400/70 after:transition-all hover:after:w-full"),
        Icon("info", cls="size-3 inline ml-1 text-muted-foreground"),
        cls="cursor-pointer inline-flex items-baseline"
    )

In [ ]:
def fasthtml_pop_text():
    return Div(Popover(PopoverFastHTML(), id="popfast", trigger_btn=PopoverTriggerButton(UnderlineText(), pid="popfast", cls="text"), side="right"), cls="w-fit")

In [ ]:
@rt
@sidebar_layout
def index():
    return Div(
        H1("BasecampUI", cls="text-2xl font-bold mt-10 mb-4"),
        
        Span("BasecampUI is a",  fasthtml_pop_text(), "UI library built on top of the excellent basecoatui css/js project. It's design goal is to create complex UI components with minimal boilerplate, while still allowing as much customization as possible."),
        Div(
            # 1. Dashed underline
            P("FastHTML", cls="underline decoration-dashed decoration-primary cursor-pointer hover:decoration-solid"),
            
            # 2. With info icon
            P(Span("FastHTML"), Icon("info", cls="size-3 inline ml-1 text-muted-foreground"), cls="cursor-pointer hover:text-primary"),
            
            # 3. Animated underline (grows on hover)
            P("FastHTML", cls="relative cursor-pointer after:absolute after:bottom-0 after:left-0 after:h-0.5 after:w-0 after:bg-primary after:transition-all hover:after:w-full"),
            
            # 4. Subtle glow/pulse
            P("FastHTML", cls="cursor-pointer text-primary animate-pulse hover:animate-none"),
            
            # 5. Highlighter effect
            P("FastHTML", cls="cursor-pointer bg-yellow-100 dark:bg-yellow-900/30 px-1 rounded hover:bg-yellow-200 dark:hover:bg-yellow-900/50"),

            P(
                Span("FastHTML", cls="relative z-10"),
                Span(
                    Icon("info", cls="size-3"),
                    cls="relative z-10 ml-1"
                ),
                cls="""cursor-pointer inline-flex items-baseline relative px-1
                    before:content-[''] before:absolute before:top-0 before:right-0 
                    before:border-8 before:border-transparent before:border-t-blue-400 before:border-r-blue-400
                    before:transition-all before:duration-300
                    hover:before:inset-0 hover:before:border-[50px] hover:before:bg-blue-400/20"""
            ),




            
            cls="flex flex-col gap-6 text-lg"
        ),
        H2("Quick start", cls="text-xl mt-5"),
        CodeHighlight("testing"),
        ChatCode("testing"),
        CopyableCode("pip install basecampui"),
        CopyableCode("from fasthtml.common import *\nfrom basecampui.all import *"),
        cls="flex flex-col w-full lg:max-w-4xl mx-auto"
    )
    

In [ ]:
pw(Span("test", cls="relative after:content-[''] after:absolute after:w-[4px] after:h-[4px] after:left-1/2 after:-top-1 after:-translate-x-1/2 after:bg-blue-500 after:rounded-lg after:transition-all  hover:cursor-pointer  hover:after:z-[-1] hover:after:bg-blue-500/40 hover:after:w-[calc(100%+10px)] hover:after:h-[calc(100%+10px)]"))